In [17]:
import os
dir_path = "val"

out_path = "imagenet_mini_path.txt"
with open(out_path, mode="w") as f:
    for current_dir, sub_dirs, files_list in os.walk(dir_path):
        for file_name in files_list:
            f.write(os.path.join(current_dir, file_name))
            f.write("\n")

In [15]:
import timm
from PIL import Image
import torch
from torchvision import transforms
from torch.nn import functional as F
import timm.models.mlp_mixer
import numpy as np
import exchange_tensor_array as exchange
import matplotlib.pyplot as plt

from pathlib import Path
from torchvision.datasets.utils import download_url
import json

import copy
#モデル作成
model = timm.create_model("gmlp_s16_224", pretrained=True)
model.eval()
print("model")

transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # 標準化する。
    ]
)
def get_classes():
    if not Path("data/imagenet_class_index.json").exists():
        # ファイルが存在しない場合はダウンロードする。
        download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

    # クラス一覧を読み込む。
    with open("data/imagenet_class_index.json") as f:
        data = json.load(f)
        class_names = [x["ja"] for x in data]
    return class_names

class_names = get_classes()

def base_model_output(_input : torch.Tensor) -> tuple:
    """画像をモデルに入力した際の一番スコアが高いラベルとそのスコアを返す"""
    base_output = model(_input)
    batch_probs = F.softmax(base_output, dim=1)
    batch_probs, batch_indices = batch_probs.sort(dim=1, descending=True)
    
    return (class_names[batch_indices[0][0]], batch_probs[0][0].item())

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model


In [33]:
result = []
with open(out_path, mode="r") as f:
    for tmp_path in f:
        img = Image.open(tmp_path.strip())
        if img.mode != "RGB":
            continue
        inputs = transform(img)
        inputs = inputs.unsqueeze(0)
        name, prob = base_model_output(inputs)
        result.append([tmp_path, name, prob])

In [34]:
result.sort(key=lambda x: x[2], reverse=True)

In [38]:
prob_path = "val_prob.txt"
with open(prob_path, mode =  "w") as f:
    for i in result:
        f.writelines(" ".join(map(str, i)) + "\n")

In [11]:
with open("C:/Users/Puran/Downloads/imagenet_mini/imagenet-mini/val/test.txt", mode="w") as f:
    f.write("test")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Puran/Downloads/imagenet_mini/imagenet-mini/val/test.txt'